In [15]:
import numpy as np
import pandas as pd
import os
from warnings import filterwarnings
filterwarnings('ignore')
pd.set_option('max_colwidth', 1000)


In [4]:
path = os.getcwd() # string representation of current directory.
print(path)

C:\Users\Ali\NLP , Naive Baise , SVM\RNN , LSTM , NER modeling , Language models


In [6]:
%cd "C:/BEPEC Python Material/DataSets , NLP datasets"

C:\BEPEC Python Material\DataSets , NLP datasets


In [7]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 8E49-65D4

 Directory of C:\BEPEC Python Material\DataSets , NLP datasets

05/04/2021  02:27 PM    <DIR>          .
05/04/2021  02:27 PM    <DIR>          ..
05/04/2021  02:27 PM        62,726,972 abcnews-date-text.csv
10/09/2020  11:03 AM            10,288 data from third party source.docx
10/09/2020  11:03 AM            10,203 data.docx
04/30/2021  10:04 AM        25,144,581 fake_news_test.csv
04/30/2021  10:03 AM        98,628,550 fake_news_train.csv
10/09/2020  11:03 AM           410,624 INX Fututre.xls
05/03/2021  10:33 AM           657,824 socrates.txt
               7 File(s)    187,589,042 bytes
               2 Dir(s)  43,155,476,480 bytes free


In [8]:
path = os.getcwd() # string representation of current directory.
print(path)

C:\BEPEC Python Material\DataSets , NLP datasets


In [35]:
for filename in os.listdir(path):
    if "abcnews" in filename:
        df = pd.read_csv(path+'/'+filename)
df.head()        

,publish_date,headline_text
0,20030219,aba decides against community broadcasting licence
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226258 entries, 0 to 1226257
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   publish_date   1226258 non-null  int64 
 1   headline_text  1226258 non-null  object
dtypes: int64(1), object(1)
memory usage: 18.7+ MB


In [37]:
## We will be taking 50000 long headlines
df = df.iloc[df['headline_text'].apply(lambda x : len(x)).sort_values(ascending = False).index].head(50000)
df.head()

,publish_date,headline_text
978636,20150914,gst changes could raise $30 billion even after low income compensation
997900,20151206,increasing gst to 15 pc could raise $27 billion grattan institute says
748563,20121213,salvation army determined to deliver on nauru despite tough conditions
876069,20140522,bland shire council revokes plans to build $5.4 million sports complex
998246,20151208,berri barmera council gets $3.5 million for multipurpose sports centre


In [39]:
df = df.reset_index()
print(len(df))
df.drop(columns = ['index' , 'publish_date'] , inplace = True)
df.head()

50000


,headline_text
0,gst changes could raise $30 billion even after low income compensation
1,increasing gst to 15 pc could raise $27 billion grattan institute says
2,salvation army determined to deliver on nauru despite tough conditions
3,bland shire council revokes plans to build $5.4 million sports complex
4,berri barmera council gets $3.5 million for multipurpose sports centre


In [42]:
headlines = []
headlines.extend(list(df["headline_text"].values))
print(len(headlines))
headlines[:5]

50000


['gst changes could raise $30 billion even after low income compensation',
 'increasing gst to 15 pc could raise $27 billion grattan institute says',
 'salvation army determined to deliver on nauru despite tough conditions',
 'bland shire council revokes plans to build $5.4 million sports complex',
 'berri barmera council gets $3.5 million for multipurpose sports centre']

### Text Cleaning

In [44]:
import string
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def clean_text(headline):
    text = "".join( word for word in headline if word not in string.punctuation ).lower()
    text = text.encode("utf8").decode("ascii", "ignore")
    return text

corpus = [ clean_text(headline) for headline in headlines ]
corpus[:5]

['gst changes could raise 30 billion even after low income compensation',
 'increasing gst to 15 pc could raise 27 billion grattan institute says',
 'salvation army determined to deliver on nauru despite tough conditions',
 'bland shire council revokes plans to build 54 million sports complex',
 'berri barmera council gets 35 million for multipurpose sports centre']

In [45]:
corpus[len(corpus)-5:]

['us broadcaster abc news reporters story about epstein was kill',
 'scotland march for independence attracts thousands post brexit',
 'artist ben quilty painting dark australian history myall creek',
 'cannabis plants budding success in secret tasmanian greenhouse',
 'fishermen stranded boat ute trailer swept away buffalo creek']

### Generating sequence of N-gram tokens

In [46]:
vocab = []
for line in corpus:
    words = line.split()
    for word in words:
        vocab.append(word)

vocablary = set(vocab)
print(len(vocablary))

35005


In [48]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [49]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word2index = tokenizer.word_index
len(word2index)

35005

In [51]:
word2index.get('and')

9

In [52]:
dictionary = {}
rev_dictionary = {}
for word, idx in word2index.items():
    dictionary[word] = idx
    rev_dictionary[idx] = word

In [53]:
from pickle import dump
# saving the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [54]:
max(rev_dictionary.keys())

35005

In [55]:
input_seqences = tokenizer.texts_to_sequences(corpus)
print(len(input_seqences))

50000


In [57]:
input_seqences[:5]

[[1484, 186, 45, 784, 1180, 420, 4164, 7, 711, 2171, 651],
 [2398, 1484, 1, 1114, 7879, 45, 784, 3664, 420, 2225, 2226, 13],
 [2572, 737, 4165, 1, 2070, 5, 1098, 213, 1282, 729],
 [14269, 1485, 60, 10008, 268, 1, 1131, 14270, 136, 1302, 3665],
 [11641, 19139, 60, 384, 3134, 136, 4, 19140, 1302, 157]]

In [72]:
input_data = []
target = []
for line in input_seqences:
    for i in range(1, len(line)-1):
        input_data.append(line[:i])
        target.append(line[i+1])
input_data[:10]

[[1484],
 [1484, 186],
 [1484, 186, 45],
 [1484, 186, 45, 784],
 [1484, 186, 45, 784, 1180],
 [1484, 186, 45, 784, 1180, 420],
 [1484, 186, 45, 784, 1180, 420, 4164],
 [1484, 186, 45, 784, 1180, 420, 4164, 7],
 [1484, 186, 45, 784, 1180, 420, 4164, 7, 711],
 [2398]]

In [69]:
target[:7]

[45, 784, 1180, 420, 4164, 7, 711]

In [60]:
MAX_LEN = 0
for seq in input_data:
    if len(seq) > MAX_LEN:
        MAX_LEN = len(seq)
MAX_LEN

13

In [62]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [63]:
input_data = pad_sequences(input_data, maxlen=MAX_LEN, padding="post", truncating="post")
print(len(input_data[0]))
input_data.shape

13


(382713, 13)

In [64]:
from tensorflow.keras.utils import to_categorical


In [70]:
vocab_size = len(word2index)+1
target = to_categorical(target, num_classes=vocab_size )
print(target.shape)
target

MemoryError: Unable to allocate 49.9 GiB for an array with shape (382713, 35006) and data type float32